## Extraction des mots clés pour de l'année 1967 

In [ ]:
import os
import yake

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [ ]:
# Année choisie
year = 1967

In [ ]:
# Lister les fichiers de cette année
data_path = '../data'
txt_path = '../data/txt'
txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]

files=txts

len(files)


In [ ]:
# Les dix premiers fichiers
files[:10]

In [ ]:
# Choisir un fichier
this_file = files[0]
this_file

In [ ]:
# Récupérer le texte du fichier
text = open(os.path.join(txt_path, this_file), 'r', encoding='utf-8').read()
text[:500]

In [ ]:
# Extraire les mots clés de ce texte
keywords = kw_extractor.extract_keywords(text)
keywords

In [ ]:
# Ne garder que les bigrammes
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

## Opération pour tous les documents de 1967


In [ ]:
for f in sorted(files)[:99]:
    text = open(os.path.join(txt_path, f), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

## Liste de stop words agrandie pour l'année 1930

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

In [ ]:
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils","elles", "il", "elle","bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville", "rossel", "dem", "si", "car", "autant", 
       "toute", "très", "bien", "aucun", "comme", "celui", "chaque", "plusieurs", 
       "toutes", "trop", "aucune", "parce", "quelques", "quel", "quelle", "quels", 
       "quelles", "lorsqu", "lorsque", "dès", "dans", "leurs", "peu", "toute", "toutes", 
       "DÈS","près", "quart","part", "ETC", "demi","bas", "grand", "demande", "vers", "avant",'vers']
sw = set(sw)

In [ ]:
# Choisir une année
year = 1967

In [ ]:
# Lister les fichiers de cette année
data_path = '../data'
txt_path = '../data/txt'
txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]
len(txts)

In [ ]:
# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in txts:
    with open(os.path.join(txt_path, txt), 'r', encoding='utf-8') as f:
        content_list.append(f.read())

In [ ]:
# Compter le nombre d'éléments (=fichiers) dans la liste
len(content_list)

In [ ]:
# Imprimer les 200 premiers caractères du contenu du premier fichier
content_list[0][0:200]

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(content_list))

In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'{year}.txt'), 'r', encoding='utf-8') as f:
    before = f.read()

before[:500]

In [ ]:
def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text(year, folder=temp_path)

In [ ]:
# Vérifier le résultat
with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

In [ ]:
frequencies = Counter(after.split())
print(frequencies.most_common(10))

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

## Reconnaisance d'entités nommées sur notre corpus pour l'année 1930

In [ ]:
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences

In [ ]:
nlp = spacy.load('fr_core_news_md')

In [ ]:
# Charger le texte
n=100000
text = open("../data/tmp/1930.txt", encoding='utf-8').read()[:n]

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

## Analyse de sentiment sur 10 phrases dans des articles de 1930

In [ ]:
import sys
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

In [ ]:
get_sentiment(" Ces banques, qui ont appauvri la paroisse autrefois aristocratique, de Sainte-Gudùle, montrent, la nuit venue, leurs façades aveugles")

In [ ]:
get_sentiment("Non! Non! Je ne veux pas! Claude! Renée! Je ne veux pas...") 

In [ ]:
get_sentiment("On est à la veille d’un certain revirement et certaines catégories Industrielles auront de la peine A échapper A un mouvement de réaction, la consommation ayant plutôt tendance à faiblir.") 

In [ ]:
get_sentiment("La mort de notre grand ami Arthur De Rudder, dit-il, a creusé un vide immense, car il fut la providence de beaucoup d’entre nous. A la lueur de cette flamme qui éclaire le sarcophage, recueillons-nous un instant, pour vénérer la mé-, moire de cet homme modeste et bon. que nous n'oublierons jamais.")

In [ ]:
get_sentiment("La remise dû tombeau à la famille fût suivie d’un long moment de silence et de recueillement. ")

In [ ]:
get_sentiment("Les bruits les plus tenaces continuent à circuler dans les milieux du Parlement, dans les salles de rédaction, partout où l’on se passionne — souvent bien à faux — pour les choses de la politique.  ")

In [ ]:
get_sentiment("La vie est chère ! Tout augmente, même le prix des volailles la veille de la Noël")

In [ ]:
get_sentiment("S’il fallait croire le quart de ce que racontent le plus sérieusement du monde des gens qui se prétendent bien informés, on serait en pleine crise.")

In [ ]:
get_sentiment("Nous commençons l'année en de bonnes dispositions peut-être trop passagères. ")

In [ ]:
get_sentiment("Il a perdu l’équilibre et est blessé à la tête mais on espère le sauver,il est tombé sous les roues de son véhicule")